In [ ]:
I am looking for the help on increasing the accuracy score for a predictive model. We already have achieved the accuracy of 72%, need to increase the same by 10% or more.

Problem Statement -  Zbay is an E-commerce website which sells a variety of products at its online platform. Zbay records user behaviour of its customers and stores it as a log. However, most of the times, users do not buy the products instantly and there is a time gap during which the customer might surf the internet and maybe visit competitor websites.